# Factor Analysis for Face Recognition

Assign a label indicating which of $M$ possible identities a face belongs to, based on a vector of grayscale pixel intensities $x$. Model the likelihood for each class using a factor analyzer:

$$
    \text{Pr}(x \ | \ w_i = k) = \text{Norm}_{x}(\mu_k, \phi_k\phi_k^T + \Sigma_k)\\[0.7em]
$$
Learn the parameters for the $k$th identity using the images of faces corresponding to that identity. Use expecation maximization to learn these parameters. Assign priors $\text{Pr}(w = k)$ according to each face's prevalence in the database. To evaluate a new face image $x_i$, compute the posterior $\text{Pr}(w_i|x_i)$.

In [ ]:
# Data:            Continuous - vectors of grayscale pixel intensities.
# World states:    Discrete - categories corresponding to face identities.
# Model:           Generative - factor analyzers each parameterized by unique means, factors, and diagonal covariance matrices.
# Learning:        Expectation maximization of each model's parameters against images of their corresponding face IDs.
# Inference:       Computation of the posterior over face IDs, given an input face image vector.

# Requirements:
#             -> [st] training_data[]    - Array with rows of grayscale face images 
#             -> [st] training_labels[]  - Vector of the faces' corresponding IDs
#             -> [fn] dnorm()            - Multivariate normal density function
#             -> [fn] posterior()        - Receives image vector, returns class probabilities
#             -> [st] priors[]           - Class priors indexed by world state
#             -> [fn] fit_priors()       - Receives training set, returns vector of class priors
#             -> [fn] likelihood()       - Receives image vector + world state, returns probability
#             -> [fn] fit_likelihoods()  - Iterates over world states and fits their corresponding likelihood functions
#             -> [fn] EM()               - Maximizes likelihood for fit_likelihoods()
#             -> [fn] E_step()           - Computes hidden variable posteriors for EM()
#             -> [fn] M_step()           - Maximizes boundary w.r.t. likelihood fn parameters for EM()
#             -> [fn] boundary()         - Computes boundary value for EM()
#             -> [st] likelihood_params[]- List containing parameters corresponding to kth likelihood function
#             -> [st] test_data[]        - Vector of test image grayscale pixel intensities

# 1. Import and flatten the data.
# 2. Fit the prior distribution     : prior = fit_prior(vector of class labels)
#    1.1. Compute and return the relative class sizes.
# 3. Fit the likelihood distribution: likelihood_parameters = fit_likelihood(matrix of image vectors, vector of class labels, n_factors)
#   3.1. Iterate over each class and maximize its images' likelihoods - Pr(x|w,th) - wr.t. th via EM.
#        -> Call: [mu_k, phi_k, covar_k] = EM(matrix of class image vectors, n_factors)
#       3.1.1. Randomly initialize vector mu_k, matrix phi_k, diag. matrix covar_k.
#       3.1.2. E_step(): Maximize the boundary w.r.t. the density functions over the hidden variable.
#       3.1.3. M_step(): Maximize the boundary's value w.r.t. mu_k, phi_k, and covar_k.
#       3.1.4. boundary(): Compute the boundary's value given the parameters and density functions.
#    3.2. Store [mu_k, phi_k, covar_k] in an array such that they are indexed by world state.
#    3.3. Return the array of parameter values.
#  4. Inference: [vector of class probabilities] = posterior(input_vec, parameter_arr)
#    4.1. Compute the product likelihood(input_vec, parameter_arr, k)*prior[k] for all world states k.
#       4.1.1. likelihood() is a multivariate normal density calculation.
#    4.2. Normalize and return this vector of unnormalized probabilities.
# 5. Display the image along with the computed class probabilities.

# What do the factors represent? Directions in which covariance is greatest among pixels.
# *Extracting the factors and plotting them as images would be interesting

In [52]:
# Import and label the training data
import glob
import numpy as np
from scipy.misc import imread, imresize

face_ids        = [fp[-6:] for fp in glob.iglob('.\\data\\face-identification\\select-faces\\*')]
face_fps        = glob.glob(face_path + '.\\data\\face-identification\\select-faces\\*\\*.jpg')
training_data   = np.array([np.ravel(imresize(imread(fp, flatten=True), size = [60, 60])) for fp in face_fps])
training_labels = [face_ids.index(name) for name in [fp.split('\\')[-2] for fp in img_fps]]